In [1]:
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

In [3]:
from thesislib.utils import pathutils

In [4]:
data_dump_file = pathutils.get_data_file("prob-synthea-1/output/data.json")
df = pd.read_json(data_dump_file)

In [6]:
classes = df['label'].unique().tolist()

In [11]:
symptoms = [val for val in df.columns if len(val) == 56 ]

In [47]:
prob_collection = {}

In [56]:
grp = df.groupby(['label'])
for itm, _df in grp.__iter__():
    prob_collection[itm] = _df[symptoms].sum()/_df.shape[0]

In [57]:
df_0 = prob_collection[0]
df_1 = prob_collection[1]

In [62]:
_1 = df_0 * df_1

In [68]:
def compute_A_between(class_key, categories, dataframe):
    """

    :param class_key:
    :param categories:
    :param dataframe:
    :return:
    """
    classes = dataframe[class_key].unique().tolist()

    # initialize the between A index
    A_between = {itm: {itm_1: 0 for itm_1 in classes} for itm in classes}

    num_categories = len(categories)

    prob_collection = {}
    # for all the conditions, compute the probabilities
    grp = dataframe.groupby([class_key])
    for itm, df in grp.__iter__():
        df = df[categories]
        prob_collection[itm] = df.sum()/df.shape[0]

    for idx in range(len(classes)):
        itm_0 = classes[idx]
        pr_0 = prob_collection[itm_0]
        for jdx in range(idx +1, len(classes)):
            itm_1 = classes[jdx]
            pr_1 = prob_collection[itm_1]

            _tmp = ((pr_0 + pr_1).sum() - 2 * (pr_0 * pr_1).sum())/num_categories
            A_between[itm_0][itm_1] = _tmp
            A_between[itm_1][itm_0] = _tmp

    return A_between

In [69]:
compute_A_between('label', symptoms, df)

{1: {1: 0,
  5: 0.12411207622698373,
  7: 0.15438180267959342,
  0: 0.17765861871904906,
  3: 0.16504078581881457,
  4: 0.20784845715147265,
  6: 0.15922564793277852,
  8: 0.18933968732119982,
  2: 0.16681141649136327},
 5: {1: 0.12411207622698373,
  5: 0,
  7: 0.14521094639222554,
  0: 0.2176018094841741,
  3: 0.1662056035983057,
  4: 0.23971888817434053,
  6: 0.15064148402811553,
  8: 0.22928287808632494,
  2: 0.15164728249676773},
 7: {1: 0.15438180267959342,
  5: 0.14521094639222554,
  7: 0,
  0: 0.20732145561559912,
  3: 0.16647349818590562,
  4: 0.23394763084518969,
  6: 0.1338187363983562,
  8: 0.21900252421774988,
  2: 0.11484248856761518},
 0: {1: 0.17765861871904906,
  5: 0.2176018094841741,
  7: 0.20732145561559912,
  0: 0,
  3: 0.2308375586580909,
  4: 0.18073916191584286,
  6: 0.20240046816999524,
  8: 0.128641451142144,
  2: 0.2237096643116519},
 3: {1: 0.16504078581881457,
  5: 0.1662056035983057,
  7: 0.16647349818590562,
  0: 0.2308375586580909,
  3: 0,
  4: 0.22854093